In [ ]:
from map_manager.map_generator import map
from map_manager.noise import toddler
from map_manager.world_encounters import *
from genetic_algorithm.crossover import * 
from genetic_algorithm.mutation import * 
import pickle
from matplotlib import colors

In [ ]:
nw = map()

nw.populate_map(n_locations=256, name="terrain",relaxed=True,k=100)
nw.populate_map(n_locations=256, name="civilisation",relaxed=True,k=100)
nw.populate_map(n_locations=256, name="threat",relaxed=True, k=100)
nw.populate_map(n_locations=256, name="faction",relaxed=True, k=100)

nw.attribute_view(seed_1=24,seed_2=32,map_name="terrain",view_name="terrain")
nw.attribute_view(seed_1=40,seed_2=50,map_name="civilisation",view_name="civilisation")
nw.attribute_view(seed_1=34,seed_2=34,map_name="threat",view_name="threat",double=False)
nw.attribute_view(seed_1=67,seed_2=89,map_name="faction",view_name="faction",double=True)

#picklefile = open('nw', 'wb')
#pickle.dump(nw, picklefile)
#picklefile.close()

#with open("nw","rb") as f:
#    nw = pickle.load(f)

nw.attribute_centroids("terrain")
nw.land_mask()


In [ ]:
pop = pack_population(nw,pop_size=100,type="camel",pack_size=50,encounter_type="friendly_animals")

pop.evolve(200,mu_p=0.22,save=False,mutation="complete")


In [ ]:
final_list = []
final_coords = []
for pack in pop.population:
    for ind in pack.pack:
        if ind.check_fitness() > -24:
            if ind.coord not in final_coords:
                final_list.append(ind)
                final_coords.append(ind.coord)



In [ ]:
a = final_coords
cmap = colors.ListedColormap(["dodgerblue","white","yellowgreen","khaki","lawngreen","slategrey","darkgreen"])
#civ = nw.views["civilisation"].copy()
ter = nw.views["terrain"].copy()
#thr = nw.views["threat"].copy()
#fac = nw.views["faction"].copy()


import matplotlib.pyplot as plt
import numpy as np 

fig, ax = plt.subplots(1,1)
fig.set_dpi(150)
fig.set_size_inches(18, 12)

lacunarity = 2

##ax[0].imshow(civ,cmap="Greens")
##ax[0].set_title("Civilisation")
##ax[0].scatter(x=[i[0] for i in a], y=[i[1] for i in a],color="Black", s=4)

ax.imshow(ter.T,cmap=cmap)
ax.set_title("Terrain")
ax.scatter(x=[i[0] for i in a], y=[i[1] for i in a],color="Black",s=10)

#for (j,i),label in np.ndenumerate(ter):
#    ["","tundra","rainforest","desert","grassland","mountain","forest"][int(label)]
#    ax.text(i,j,label,ha='center',va='center')





#ax[2].imshow(thr,cmap="Reds")
#ax[2].set_title("Threat")
#ax[2].scatter(x=[i[0] for i in a], y=[i[1] for i in a],color="Black")


#ax[3].imshow(fac,cmap="Greys")
#ax[3].set_title("Factions")
#ax[3].scatter(x=[i[0] for i in a], y=[i[1] for i in a],color="Black")



[[i/6 for i in range(0,5)],

In [ ]:
!pip install svgpath2mpl
!pip install svgpathtools

import matplotlib as mpl
from svgpathtools import svg2paths
from svgpath2mpl import parse_path

planet_path, attributes = svg2paths('/Users/cetiners/Desktop/Thesis/human_error/camel-svgrepo-com.svg')

planet_marker = parse_path(attributes[0]['d'])


In [ ]:
planet_marker.vertices -= planet_marker.vertices.mean(axis=0)

planet_marker = planet_marker.transformed(mpl.transforms.Affine2D().rotate_deg(180))
planet_marker = planet_marker.transformed(mpl.transforms.Affine2D().scale(-1,1))